In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopandas import GeoDataFrame, GeoSeries
from geopandas.tools import sjoin
%matplotlib inline

In [3]:
# Load in data across all years to split across values
suspensions_df = pd.read_csv("suspension_data/cps_suspension_homeroom.csv")
suspensions_df[:5]

,Unnamed: 0,School Year,School ID,School Name,Homeroom,Count of Suspension Events,Dummy Room number to remove names,Original Homeroom
0,0,2010-2011,400018,AUSTIN BUS & ENTRP CONTR HS,A101,*,NaN,A101
1,1,2010-2011,400018,AUSTIN BUS & ENTRP CONTR HS,A102,*,NaN,A102
2,2,2010-2011,400022,CHGO ARTS CONTR HS,A103,*,NaN,A103
3,3,2010-2011,400040,FRAZIER CONTR,A104,*,NaN,A104
4,4,2010-2011,400043,HOPE CONTR ES,A110,*,NaN,A110


In [4]:
# Process suspension events where value is * to 0
suspensions_df["Count of Suspension Events"] = pd.to_numeric(suspensions_df["Count of Suspension Events"], errors='coerce')
suspensions_df["Count of Suspension Events"].fillna(0, inplace=True)
suspensions_df["Count of Suspension Events"].sort_values(ascending=False)[:10]

32507    3425
32506    2892
32505    2310
32504    2219
32503    1614
32502    1526
32501    1483
32500    1402
32499    1387
32498    1346
Name: Count of Suspension Events, dtype: float64

In [5]:
# Split data by year
suspensions_1011 = suspensions_df[suspensions_df["School Year"] == "2010-2011"]
suspensions_1112 = suspensions_df[suspensions_df["School Year"] == "2011-2012"]
suspensions_1213 = suspensions_df[suspensions_df["School Year"] == "2012-2013"]
suspensions_1314 = suspensions_df[suspensions_df["School Year"] == "2013-2014"]
suspensions_1415 = suspensions_df[suspensions_df["School Year"] == "2014-2015"]

suspensions_1011[:10]

,Unnamed: 0,School Year,School ID,School Name,Homeroom,Count of Suspension Events,Dummy Room number to remove names,Original Homeroom
0,0,2010-2011,400018,AUSTIN BUS & ENTRP CONTR HS,A101,0,NaN,A101
1,1,2010-2011,400018,AUSTIN BUS & ENTRP CONTR HS,A102,0,NaN,A102
2,2,2010-2011,400022,CHGO ARTS CONTR HS,A103,0,NaN,A103
3,3,2010-2011,400040,FRAZIER CONTR,A104,0,NaN,A104
4,4,2010-2011,400043,HOPE CONTR ES,A110,0,NaN,A110
5,5,2010-2011,400068,PLATO CONTR,A201,0,NaN,A201
6,6,2010-2011,400091,CHGO ADV CONTR HS,A202,0,NaN,A202
7,7,2010-2011,400095,GARFIELD PARK CONTR,A203,0,NaN,A203
8,8,2010-2011,400095,GARFIELD PARK CONTR,A210,0,NaN,A210
9,9,2010-2011,400095,GARFIELD PARK CONTR,A301,0,NaN,A301


In [6]:
# Load race data for each school year
race_1011 = pd.read_csv("race_data/cps_race_data_1011.csv")
race_1112 = pd.read_csv("race_data/cps_race_data_1112.csv")
race_1213 = pd.read_csv("race_data/cps_race_data_1213.csv")
race_1314 = pd.read_csv("race_data/cps_race_data_1314.csv")
race_1415 = pd.read_csv("race_data/cps_race_data_1415.csv")

# Create combination Black/Hispanic variable for each (more traditionally under-resourced)
race_1011["Black/Hispanic Pct"] = race_1011["African American Pct"] + race_1011["Hispanic Pct"]
race_1112["Black/Hispanic Pct"] = race_1112["African American Pct"] + race_1112["Hispanic Pct"]
race_1213["Black/Hispanic Pct"] = race_1213["African American Pct"] + race_1213["Hispanic Pct"]
race_1314["Black/Hispanic Pct"] = race_1314["African American Pct"] + race_1314["Hispanic Pct"]
race_1415["Black/Hispanic Pct"] = race_1415["African American Pct"] + race_1415["Hispanic Pct"]

race_1415[:10]

,Network,School ID,Education Units,Total,White No,White Pct,African American No,African American Pct,Asian/Pac Islander No,Asian/Pac Islander Pct,...,Asian No,Asian Pct,Hawaiian Pac Islander No,Hawaiian Pac Islander Pct,Na No,Na Pct,Unnamed: 22,Unnamed: 23,Unnamed: 24,Black/Hispanic Pct
0,Charter,400010,ACE TECH HS,439,6,1.4,370,84.3,0,0.0,...,0,0.0,0,0.0,1,0.2,NaN,NaN,NaN,98.2
1,Network 13,609772,ADDAMS,894,14,1.6,16,1.8,0,0.0,...,0,0.0,0,0.0,1,0.1,NaN,NaN,NaN,98.2
2,Network 4,609773,AGASSIZ,527,186,35.3,140,26.6,0,0.0,...,21,4.0,1,0.2,1,0.2,NaN,NaN,NaN,56.0
3,Charter,400060,AHS - PASSAGES,456,29,6.4,209,45.8,0,0.0,...,102,22.4,3,0.7,3,0.7,NaN,NaN,NaN,65.5
4,Service Leadership Academies,610513,AIR FORCE HS,328,36,11.0,139,42.4,0,0.0,...,12,3.7,0,0.0,1,0.3,NaN,NaN,NaN,84.8
5,Network 1,610212,ALBANY PARK,286,17,5.9,9,3.1,0,0.0,...,30,10.5,0,0.0,0,0.0,NaN,NaN,NaN,81.1
6,Network 4,609774,ALCOTT ES,576,376,65.3,45,7.8,1,0.2,...,41,7.1,1,0.2,12,2.1,NaN,NaN,NaN,19.1
7,Network 4,610524,ALCOTT HS,308,28,9.1,63,20.5,0,0.0,...,4,1.3,0,0.0,4,1.3,NaN,NaN,NaN,86.7
8,Network 13,609848,ALDRIDGE,194,0,0.0,193,99.5,0,0.0,...,0,0.0,0,0.0,0,0.0,NaN,NaN,NaN,100.0
9,Charter,400012,AMANDLA,323,1,0.3,316,97.8,0,0.0,...,0,0.0,0,0.0,2,0.6,NaN,NaN,NaN,98.7


In [7]:
# Group suspensions by school for each school year
schools_1011 = suspensions_1011.groupby(["School Name", "School ID"])["Count of Suspension Events"].sum()
schools_1112 = suspensions_1112.groupby(["School Name", "School ID"])["Count of Suspension Events"].sum()
schools_1213 = suspensions_1213.groupby(["School Name", "School ID"])["Count of Suspension Events"].sum()
schools_1314 = suspensions_1314.groupby(["School Name", "School ID"])["Count of Suspension Events"].sum()
schools_1415 = suspensions_1415.groupby(["School Name", "School ID"])["Count of Suspension Events"].sum()

# Reset indices to re-sort
schools_1011 = schools_1011.reset_index()
schools_1112 = schools_1112.reset_index()
schools_1213 = schools_1213.reset_index()
schools_1314 = schools_1314.reset_index()
schools_1415 = schools_1415.reset_index()

# Sort each dataframe by suspensions
schools_1011.sort_values(by="Count of Suspension Events",ascending=False,inplace=True)
schools_1112.sort_values(by="Count of Suspension Events",ascending=False,inplace=True)
schools_1213.sort_values(by="Count of Suspension Events",ascending=False,inplace=True)
schools_1314.sort_values(by="Count of Suspension Events",ascending=False,inplace=True)
schools_1415.sort_values(by="Count of Suspension Events",ascending=False,inplace=True)

# Print for 10-11
schools_1011[:10]

,School Name,School ID,Count of Suspension Events
183,FENGER HS,609705,3463
504,STEINMETZ HS,609732,2561
292,KENNEDY HS,609718,2534
181,FARRAGUT HS,609704,2489
284,JULIAN HS,609762,2379
155,DUNBAR HS,609676,2276
400,ORR HS,610389,2067
235,HARLAN HS,609710,1997
338,MARSHALL HS,609723,1935
332,MANLEY HS,609722,1900


In [8]:
# Merge yearly race data with yearly suspensions on school
suspensions_race_1011 = pd.merge(schools_1011, race_1011, on="School ID")
suspensions_race_1112 = pd.merge(schools_1112, race_1112, on="School ID")
suspensions_race_1213 = pd.merge(schools_1213, race_1213, on="School ID")
suspensions_race_1314 = pd.merge(schools_1314, race_1314, on="School ID")
suspensions_race_1415 = pd.merge(schools_1415, race_1415, on="School ID")

In [9]:
suspensions_race_1011[:10]

,School Name,School ID,Count of Suspension Events,Area,Unit,School,Total,White No,White Pct,African American No,...,Hispanic Pct,Multi-Racial No,Multi-Racial Pct,Asian No,Asian Pct,Hawaiian Pac Islander No,Hawaiian Pac Islander Pct,Na No,Na Pct,Black/Hispanic Pct
0,FENGER HS,609705,3463,29,1310,Fenger HS,784,0,0.0,773,...,1.0,1,0.1,0,0.0,1,0.1,0,0.0,99.6
1,STEINMETZ HS,609732,2561,26,1560,Steinmetz HS,"1,889",189,10.0,316,...,70.0,7,0.4,21,1.1,3,0.2,5,0.3,86.7
2,KENNEDY HS,609718,2534,24,1420,Kennedy HS,"1,580",432,27.3,105,...,63.7,4,0.3,15,0.9,2,0.1,1,0.1,70.3
3,FARRAGUT HS,609704,2489,26,1300,Farragut HS,"1,598",7,0.4,226,...,84.1,3,0.2,0,0.0,1,0.1,0,0.0,98.2
4,JULIAN HS,609762,2379,24,1870,Julian HS,"1,224",0,0.0,"1,219",...,0.2,2,0.2,0,0.0,0,0.0,0,0.0,99.8
5,DUNBAR HS,609676,2276,19,1030,Dunbar Voc HS,"1,507",2,0.1,"1,475",...,1.1,8,0.5,2,0.1,1,0.1,1,0.1,99.0
6,ORR HS,610389,2067,28,1830,Orr Academy HS,956,2,0.2,812,...,9.8,47,4.9,0,0.0,0,0.0,0,0.0,94.7
7,HARLAN HS,609710,1997,23,1350,Harlan HS,"1,260",0,0.0,"1,246",...,0.7,5,0.4,0,0.0,0,0.0,0,0.0,99.6
8,MARSHALL HS,609723,1935,29,1470,Marshall HS,772,0,0.0,765,...,0.8,1,0.1,0,0.0,0,0.0,0,0.0,99.9
9,MANLEY HS,609722,1900,21,1460,Manley HS,650,0,0.0,649,...,0.2,0,0.0,0,0.0,0,0.0,0,0.0,100.0


In [10]:
suspensions_race_1415[:10]

,School Name,School ID,Count of Suspension Events,Network,Education Units,Total,White No,White Pct,African American No,African American Pct,...,Asian No,Asian Pct,Hawaiian Pac Islander No,Hawaiian Pac Islander Pct,Na No,Na Pct,Unnamed: 22,Unnamed: 23,Unnamed: 24,Black/Hispanic Pct
0,DUNBAR HS,609676,3676,Network 9,DUNBAR HS,761,3,0.4,744,97.8,...,1,0.1,0,0.0,0,0.0,NaN,NaN,NaN,99.2
1,ORR HS,610389,3484,AUSL,ORR HS,405,1,0.2,347,85.7,...,0,0.0,0,0.0,0,0.0,NaN,NaN,NaN,99.5
2,BOWEN HS,610323,2797,OS4,BOWEN HS,359,2,0.6,271,75.5,...,0,0.0,0,0.0,1,0.3,NaN,NaN,NaN,97.5
3,STEINMETZ COLLEGE PREP HS,609732,2639,Network 3,STEINMETZ HS,"1,545",137,8.9,220,14.2,...,33,2.1,1,0.1,1,0.1,NaN,NaN,NaN,87.3
4,PHILLIPS HS,609727,2016,AUSL,PHILLIPS HS,643,2,0.3,627,97.5,...,2,0.3,1,0.2,0,0.0,NaN,NaN,NaN,98.6
5,LAKE VIEW HS,609719,1789,Network 2,LAKE VIEW HS,"1,289",161,12.5,163,12.6,...,49,3.8,2,0.2,0,0.0,NaN,NaN,NaN,81.0
6,WASHINGTON HS,609739,1675,Network 13,WASHINGTON HS,"1,433",81,5.7,92,6.4,...,1,0.1,0,0.0,0,0.0,NaN,NaN,NaN,93.4
7,FENGER HS,609705,1464,Network 13,FENGER HS,271,1,0.4,265,97.8,...,0,0.0,1,0.4,0,0.0,NaN,NaN,NaN,98.9
8,MARSHALL HS,609723,1449,ISP,MARSHALL HS,425,0,0.0,416,97.9,...,0,0.0,0,0.0,0,0.0,NaN,NaN,NaN,99.3
9,SENN HS,609730,1173,Network 2,SENN HS,"1,365",135,9.9,380,27.8,...,203,14.9,18,1.3,1,0.1,NaN,NaN,NaN,70.6


In [11]:
suspensions_race_1011.describe()

,School ID,Count of Suspension Events,White Pct,African American Pct,Native American/Alaskan Pct,Asian/Pac Islander No,Asian/Pac Islander Pct,Hispanic Pct,Multi-Racial Pct,Asian Pct,Hawaiian Pac Islander No,Hawaiian Pac Islander Pct,Na Pct,Black/Hispanic Pct
count,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000,577.000000
mean,607141.833622,164.350087,7.555459,53.584402,0.326516,0.355286,0.040901,34.546794,1.001560,2.535529,0.668977,0.104679,0.304159,88.131196
std,24577.618119,378.655304,14.886885,42.286381,0.473324,2.483336,0.241791,36.372689,1.601783,7.284804,1.947120,0.280121,1.050124,19.878600
min,400018.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.600000
25%,609861.000000,13.000000,0.000000,6.600000,0.000000,0.000000,0.000000,1.100000,0.000000,0.000000,0.000000,0.000000,0.000000,89.000000
50%,610048.000000,53.000000,0.700000,54.600000,0.200000,0.000000,0.000000,20.500000,0.400000,0.000000,0.000000,0.000000,0.000000,97.800000
75%,610220.000000,132.000000,5.500000,97.800000,0.500000,0.000000,0.000000,70.700000,1.200000,1.300000,1.000000,0.100000,0.200000,99.400000
max,610544.000000,3463.000000,77.300000,100.000000,3.400000,51.000000,3.700000,99.600000,10.000000,92.800000,25.000000,3.600000,10.600000,100.000000


In [13]:
suspensions_race_1112.describe()

,School ID,Count of Suspension Events,CPS Unit,White Pct,African American Pct,Asian/Pac Islander No,Asian/Pac Islander Pct,Native American/Alaskan Pct,Hispanic Pct,Multi-Racial Pct,Asian Pct,Hawaiian Pac Islander No,Hawaiian Pac Islander Pct,Na Pct,Unnamed: 23,Unnamed: 24,Unnamed: 25,Black/Hispanic Pct
count,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,572.000000,0,0,0,572.000000
mean,607120.377622,147.274476,4612.000000,7.854720,53.422028,0.160839,0.023601,0.290559,34.306294,1.006643,2.412238,0.777972,0.109790,0.573427,NaN,NaN,NaN,87.728322
std,24684.084349,371.736541,2082.896149,15.371962,42.184246,0.968147,0.164713,0.385165,36.131701,1.511123,6.268135,2.670066,0.306054,1.491714,NaN,NaN,NaN,20.048827
min,400018.000000,0.000000,1010.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,16.600000
25%,609863.750000,0.000000,2845.000000,0.000000,6.550000,0.000000,0.000000,0.000000,1.400000,0.100000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,87.550000
50%,610052.500000,36.000000,4590.000000,0.750000,53.800000,0.000000,0.000000,0.100000,20.150000,0.400000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,97.600000
75%,610219.250000,115.250000,6322.500000,5.825000,97.700000,0.000000,0.000000,0.500000,68.300000,1.200000,1.400000,1.000000,0.100000,0.425000,NaN,NaN,NaN,99.300000
max,610556.000000,3270.000000,8686.000000,80.000000,100.000000,17.000000,2.600000,2.500000,99.700000,9.500000,57.800000,37.000000,4.100000,13.100000,NaN,NaN,NaN,100.100000


In [14]:
suspensions_race_1213.describe()

,School ID,Count of Suspension Events,White Pct,African American Pct,Asian/Pac Islander No,Asian/Pac Islander Pct,Native American/Alaskan Pct,Hispanic Pct,Multi-Racial Pct,Asian Pct,Hawaiian Pac Islander No,Hawaiian Pac Islander Pct,Na Pct,Unnamed: 22,Unnamed: 23,Unnamed: 24,Black/Hispanic Pct
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,0,0,0,569.000000
mean,607473.760984,154.690685,8.226362,51.986819,0.145870,0.019684,0.296485,35.019156,1.032162,2.566784,0.871705,0.119332,0.729350,NaN,NaN,NaN,87.005975
std,23170.383924,357.232727,15.770126,42.279416,0.770144,0.116428,0.399676,36.428350,1.544425,6.533947,3.265169,0.358566,1.871788,NaN,NaN,NaN,20.804721
min,400018.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,15.800000
25%,609865.000000,10.000000,0.000000,5.800000,0.000000,0.000000,0.000000,1.500000,0.100000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,84.400000
50%,610048.000000,37.000000,0.900000,49.300000,0.000000,0.000000,0.200000,20.600000,0.400000,0.000000,0.000000,0.000000,0.100000,NaN,NaN,NaN,97.500000
75%,610216.000000,123.000000,6.400000,97.300000,0.000000,0.000000,0.500000,71.100000,1.200000,1.600000,1.000000,0.100000,0.600000,NaN,NaN,NaN,99.200000
max,610559.000000,3184.000000,79.900000,100.000000,14.000000,2.100000,4.200000,99.200000,10.000000,60.000000,57.000000,5.900000,20.700000,NaN,NaN,NaN,100.000000


In [15]:
suspensions_race_1314.describe()

,School ID,Count of Suspension Events,White Pct,African American Pct,Asian/Pac Islander No,Asian/Pac Islander Pct,Native American/Alaskan Pct,Hispanic Pct,Multi-Racial Pct,Asian Pct,Hawaiian Pac Islander No,Hawaiian Pac Islander Pct,Na Pct,Unnamed: 22,Unnamed: 23,Unnamed: 24,Black/Hispanic Pct
count,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,0,0,0,521.000000
mean,607229.771593,139.040307,8.858541,48.728407,0.069098,0.008637,0.271785,37.437236,1.074856,2.991171,1.034549,0.136276,0.490595,NaN,NaN,NaN,86.165643
std,24199.067823,375.683117,16.304837,42.255666,0.381073,0.053550,0.332170,36.753694,1.536367,7.715484,3.160872,0.352639,1.305597,NaN,NaN,NaN,21.708445
min,400018.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,14.600000
25%,609853.000000,0.000000,0.100000,4.900000,0.000000,0.000000,0.000000,2.000000,0.200000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,83.800000
50%,610034.000000,30.000000,1.100000,37.200000,0.000000,0.000000,0.200000,25.000000,0.500000,0.200000,0.000000,0.000000,0.000000,NaN,NaN,NaN,97.400000
75%,610208.000000,86.000000,8.100000,96.800000,0.000000,0.000000,0.400000,77.000000,1.300000,2.000000,1.000000,0.200000,0.400000,NaN,NaN,NaN,99.100000
max,610573.000000,2769.000000,82.000000,100.000000,4.000000,0.800000,1.900000,99.700000,10.500000,83.500000,45.000000,5.000000,12.400000,NaN,NaN,NaN,100.000000


In [16]:
suspensions_race_1415.describe()

,School ID,Count of Suspension Events,White Pct,African American Pct,Asian/Pac Islander No,Asian/Pac Islander Pct,Native American/Alaskan No,Native American/Alaskan Pct,Hispanic Pct,Multi-Racial No,...,Asian No,Asian Pct,Hawaiian Pac Islander No,Hawaiian Pac Islander Pct,Na No,Na Pct,Unnamed: 22,Unnamed: 23,Unnamed: 24,Black/Hispanic Pct
count,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,...,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,0,0,0,501.000000
mean,607534.924152,92.630739,8.896008,48.165669,0.017964,0.002395,1.584830,0.196208,38.131936,7.235529,...,26.021956,3.060279,1.153693,0.158882,2.307385,0.346707,NaN,NaN,NaN,86.297605
std,22865.852988,352.848712,16.489117,41.943613,0.172269,0.021777,2.485013,0.266844,36.572676,12.885822,...,76.742254,7.882880,3.131506,0.366641,7.315007,0.950186,NaN,NaN,NaN,21.753514
min,400018.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,13.600000
25%,609852.000000,0.000000,0.200000,5.000000,0.000000,0.000000,0.000000,0.000000,2.500000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,84.400000
50%,610036.000000,0.000000,1.000000,34.500000,0.000000,0.000000,1.000000,0.100000,25.500000,2.000000,...,1.000000,0.200000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,97.600000
75%,610207.000000,32.000000,8.200000,96.400000,0.000000,0.000000,2.000000,0.300000,76.900000,7.000000,...,13.000000,2.300000,1.000000,0.200000,2.000000,0.300000,NaN,NaN,NaN,99.200000
max,610570.000000,3676.000000,84.000000,100.000000,3.000000,0.300000,19.000000,1.800000,99.500000,109.000000,...,927.000000,83.800000,39.000000,4.400000,76.000000,11.300000,NaN,NaN,NaN,100.000000
